## Read data from database

In [1]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [3]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [4]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [5]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# table_suffix = ["0","3","9","A","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where (CNT_CODE = 1 or MEM_EXECUTE = 1) and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201704')
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

--- 3.8712735176086426 seconds ---
--- 4.044234037399292 seconds ---
--- 3.291417121887207 seconds ---
--- 3.3292155265808105 seconds ---
--- 3.087956428527832 seconds ---
--- 3.0757627487182617 seconds ---
--- 3.091250419616699 seconds ---
--- 3.033792734146118 seconds ---
--- 2.9319264888763428 seconds ---
--- 3.1569557189941406 seconds ---


KeyboardInterrupt: 

In [ ]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# table_suffix = ["0","3","9","A","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where (CNT_CODE = 1 or MEM_EXECUTE = 1) and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201705')
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

## Check and split data

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines >= 4 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines >= 4 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

In [ ]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

In [ ]:
def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

## Autoencoder

In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [ ]:
import keras
import numpy as np

max_length = 30000

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, batch_size=32, dim=max_length, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, Y = self.__data_generation(list_IDs_temp)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > self.dim:
                bytes_data = [int(single_byte) for single_byte in in_file.read(self.dim)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data

#         X = X.reshape((-1, 100, 100, 1)) / 255.0
#         Y = X.reshape((-1, 10000)) / 255.0
        Y = X.reshape((-1, max_length, 1)) / 255.0
        return Y, Y

In [ ]:
import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Conv2D, Multiply, GlobalMaxPooling1D, Dropout, Activation
from keras.layers import UpSampling2D, Flatten, merge, MaxPooling2D, MaxPooling1D, UpSampling1D
from keras.models import load_model, Model
from keras.layers import merge, Dropout, BatchNormalization, Maximum, Add
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split

class Autoencoder():
    def __init__(self, autoencoder_name, encoder_name):
        self.autoencoder_name = autoencoder_name
        self.encoder_name = encoder_name
        self.autoencoder = None
        self.encoder = None
        self.start_time = time.time()

    def get_model(self):
# Deep
        input_sequence = Input(shape=(max_length, 1));
    
        x = Conv1D(32, 5, padding='same')(input_sequence)
        x = Conv1D(32, 5, padding='causal', dilation_rate=1)(x)
        x = MaxPooling1D(5, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=2)(x)
        x = MaxPooling1D(5, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=4)(x)
        x = MaxPooling1D(3, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=8)(x)
        x = MaxPooling1D(2, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=16)(x)
        x = MaxPooling1D(2, padding='same')(x)
        encoded = Dense(1, activation='sigmoid')(x)
        
        x = Conv1D(32, 3, padding='same')(encoded)
        x = Conv1D(32, 3, padding='causal', dilation_rate=16)(x)
        x = UpSampling1D(2)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=8)(x)
        x = UpSampling1D(2)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=4)(x)
        x = UpSampling1D(3)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=2)(x)
        x = UpSampling1D(5)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 5, padding='same')(x)
        x = Conv1D(32, 5, padding='causal', dilation_rate=1)(x)
        x = UpSampling1D(5)(x)
        decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
        
## Deep
#         x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
#         x = MaxPooling2D((2, 2), padding='same')(x)
#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
#         x = MaxPooling2D((2, 2), padding='same')(x)
#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
#         x = MaxPooling2D((5, 5), padding='same')(x)
#         encoded = Dense(2, activation='sigmoid')(x)
        
#         # at this point the representation is (4, 4, 8) i.e. 128-dimensional

#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
#         x = UpSampling2D((5, 5))(x)
#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
#         x = UpSampling2D((2, 2))(x)
#         x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#         x = UpSampling2D((2, 2))(x)
#         decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

## Base
#         x = Conv2D(32, 4, activation='relu', padding='same')(input_img)
#         x = MaxPooling2D(25, padding='same')(x)
#         encoded = Dense(1, activation='sigmoid')(x)

#         x = Conv2D(32, 4, activation='relu', padding='same')(encoded)
#         x = UpSampling2D(25)(x)
#         decoded = Conv2D(1, 4, activation='sigmoid', padding='same')(x)

        self.autoencoder = Model(inputs=input_sequence, outputs=decoded)
        self.encoder = Model(inputs=input_sequence, outputs=encoded)

        self.autoencoder.compile(loss='mean_squared_error', optimizer='adam')
#         self.autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
        self.autoencoder.summary()

    def train(self, train_df, max_epoch, batch_size=32):
        self.get_model()
        partition_train, partition_validation = train_test_split(range(len(train_df)), test_size=0.05,
                                                                 random_state=1234)
        print('Length of the train: ', len(partition_train))
        print('Length of the validation: ', len(partition_validation))

        #         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/autoencoder/models/"+ str(self.start_time) +"-{epoch:04d}-{val_loss:.5f}_12_3_a_0.h5"
        #         early_stopping = EarlyStopping("val_loss", patience=2, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point]
        
        model_dir = '/home/zhaoqi/autoencoder/models/'
        f_name = '1533708921.8992589-0056-0.06208_12_3_a_0.h5'
        self.autoencoder = load_model(model_dir + f_name)

        # Generators
        training_generator = DataGenerator(partition_train, train_df, batch_size)
        validation_generator = DataGenerator(partition_validation, train_df, batch_size)

        self.autoencoder.fit_generator(generator=training_generator,
                                       validation_data=validation_generator,
                                       use_multiprocessing=True,
                                       epochs=max_epoch,
                                       workers=6,
                                       callbacks=callbacks_list)
        self.autoencoder.save(self.autoencoder_name)
        self.encoder.save(self.encoder_name)
        
        
autoencoder = Autoencoder('autoencoder.h5', 'encoder.h5')
autoencoder.train(x_train, max_epoch=64, batch_size=32)

In [13]:
from keras.models import Model

model_dir = '/home/zhaoqi/autoencoder/models/'
f_name = '1533812481.707364-0053-0.06197_12_3_a_0.h5'
c_model = load_model(model_dir + f_name)

model_f = Model(c_model.input, c_model.layers[-21].output)

model_f.summary()

train_generator = DataGenerator(range(len(x_train)), x_train, 16, max_length, False)
malcon_train_x = model_f.predict_generator(generator=train_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

val_generator = DataGenerator(range(len(x_val)), x_val, 16, max_length, False)
malcon_val_x = model_f.predict_generator(generator=val_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

test_generator = DataGenerator(range(len(x_test)), x_test, 16, max_length, False)
malcon_test_x = model_f.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 30000, 1)          0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 30000, 32)         192       
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 30000, 32)         5152      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 6000, 32)          0         
_________________________________________________________________
activation_25 (Activation)   (None, 6000, 32)          0         
_________________________________________________________________
conv1d_66 (Conv1D)           (None, 6000, 32)          3104      
_________________________________________________________________
conv1d_67 (Conv1D)           (None, 6000, 32)          3104      
__________

In [14]:
malcon_train_x = malcon_train_x.reshape(len(malcon_train_x), -1)
malcon_val_x = malcon_val_x.reshape(len(malcon_val_x), -1)
malcon_test_x = malcon_test_x.reshape(len(malcon_test_x), -1)

In [15]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

In [16]:
def get_max_feature(origin_data, feature_data):
    feature_data = pd.DataFrame(feature_data)
    feature_data['mw_file_hash'] = origin_data.iloc[0:len(feature_data)][['mw_file_hash']]
    feature_data = feature_data.groupby('mw_file_hash').max().merge(origin_data[['mw_file_hash','mw_em_f','mw_num_engines']], how='inner', on='mw_file_hash')
    tmp_ember = get_ember_feature(feature_data[['mw_em_f']])
    tmp_label = feature_data[['mw_num_engines']].mw_num_engines
    tmp_data = feature_data.drop(['mw_num_engines', 'mw_em_f', 'mw_file_hash'], axis=1)
    return tmp_data, tmp_label, tmp_ember

In [17]:
max_train_data, max_train_label, max_train_ember = get_max_feature(x_train, malcon_train_x)
max_val_data, max_val_label, max_val_ember = get_max_feature(x_val, malcon_val_x)
max_test_data, max_test_label, max_test_ember = get_max_feature(x_test, malcon_test_x)

In [18]:
def merge_feature(m_data, e_data):
    num = len(m_data)
    m_x = np.zeros((num, 100+2351), dtype=float)
    
    for index in range(num):
        m_x[index, 0:100] = m_data.iloc[index]
        m_x[index, 100:100+515] = e_data[index][0:515]
        m_x[index, 100+515:100+2351-96] = e_data[index][515+96:]
        m_x[index, 100+2351-96:] = e_data[index][515:515+96]
    return m_x

In [19]:
merge_train_x = merge_feature(max_train_data, max_train_ember)
merge_val_x = merge_feature(max_val_data, max_val_ember)
merge_test_x = merge_feature(max_test_data, max_test_ember)

In [20]:
def get_model(x_train, y_train, x_val, y_val):
    params = {'application': 'binary'}
    lgbm_dataset = lgb.Dataset(x_train, y_train.ravel())
    valid_sets = lgb.Dataset(x_val, y_val.ravel())

    model = lgb.train(params, lgbm_dataset, valid_sets=valid_sets, num_boost_round=10000, early_stopping_rounds=10)
    y_pred = model.predict(x_val, num_iteration=model.best_iteration)
    
    loss = log_loss(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
#     model.save_model(file_path + "-%04d-%.5f-%.5f.h5" % (model.best_iteration, loss, acc),
#                      num_iteration=model.best_iteration)
    print("val loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)
        
    return model

In [21]:
model_m = get_model(merge_train_x[:,:100+2351-96], max_train_label, merge_val_x[:,:100+2351-96], max_val_label )

[1]	valid_0's binary_logloss: 0.585915
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.524647
[3]	valid_0's binary_logloss: 0.473121
[4]	valid_0's binary_logloss: 0.43013
[5]	valid_0's binary_logloss: 0.392125
[6]	valid_0's binary_logloss: 0.358889
[7]	valid_0's binary_logloss: 0.330613
[8]	valid_0's binary_logloss: 0.305607
[9]	valid_0's binary_logloss: 0.28394
[10]	valid_0's binary_logloss: 0.2648
[11]	valid_0's binary_logloss: 0.246593
[12]	valid_0's binary_logloss: 0.230347
[13]	valid_0's binary_logloss: 0.215479
[14]	valid_0's binary_logloss: 0.203112
[15]	valid_0's binary_logloss: 0.191014
[16]	valid_0's binary_logloss: 0.180883
[17]	valid_0's binary_logloss: 0.171198
[18]	valid_0's binary_logloss: 0.163107
[19]	valid_0's binary_logloss: 0.154856
[20]	valid_0's binary_logloss: 0.147486
[21]	valid_0's binary_logloss: 0.141149
[22]	valid_0's binary_logloss: 0.135079
[23]	valid_0's binary_logloss: 0.129984
[24]	valid_0's binary_logloss: 

[199]	valid_0's binary_logloss: 0.0248785
[200]	valid_0's binary_logloss: 0.0247787
[201]	valid_0's binary_logloss: 0.0247177
[202]	valid_0's binary_logloss: 0.0246383
[203]	valid_0's binary_logloss: 0.0245321
[204]	valid_0's binary_logloss: 0.0244251
[205]	valid_0's binary_logloss: 0.0243613
[206]	valid_0's binary_logloss: 0.0243011
[207]	valid_0's binary_logloss: 0.0242216
[208]	valid_0's binary_logloss: 0.0241532
[209]	valid_0's binary_logloss: 0.0240896
[210]	valid_0's binary_logloss: 0.0239984
[211]	valid_0's binary_logloss: 0.0239404
[212]	valid_0's binary_logloss: 0.0238974
[213]	valid_0's binary_logloss: 0.0238397
[214]	valid_0's binary_logloss: 0.0238273
[215]	valid_0's binary_logloss: 0.0237796
[216]	valid_0's binary_logloss: 0.0236321
[217]	valid_0's binary_logloss: 0.0235224
[218]	valid_0's binary_logloss: 0.0234044
[219]	valid_0's binary_logloss: 0.0233411
[220]	valid_0's binary_logloss: 0.0232955
[221]	valid_0's binary_logloss: 0.023265
[222]	valid_0's binary_logloss: 0.0

[395]	valid_0's binary_logloss: 0.0169777
[396]	valid_0's binary_logloss: 0.0169104
[397]	valid_0's binary_logloss: 0.016882
[398]	valid_0's binary_logloss: 0.016931
[399]	valid_0's binary_logloss: 0.0169063
[400]	valid_0's binary_logloss: 0.0168715
[401]	valid_0's binary_logloss: 0.0168818
[402]	valid_0's binary_logloss: 0.0168607
[403]	valid_0's binary_logloss: 0.0168531
[404]	valid_0's binary_logloss: 0.0168212
[405]	valid_0's binary_logloss: 0.0167939
[406]	valid_0's binary_logloss: 0.0167703
[407]	valid_0's binary_logloss: 0.0167656
[408]	valid_0's binary_logloss: 0.0167313
[409]	valid_0's binary_logloss: 0.0166868
[410]	valid_0's binary_logloss: 0.0166681
[411]	valid_0's binary_logloss: 0.0166561
[412]	valid_0's binary_logloss: 0.0166235
[413]	valid_0's binary_logloss: 0.0166188
[414]	valid_0's binary_logloss: 0.0166165
[415]	valid_0's binary_logloss: 0.0166037
[416]	valid_0's binary_logloss: 0.0165771
[417]	valid_0's binary_logloss: 0.016593
[418]	valid_0's binary_logloss: 0.016

[592]	valid_0's binary_logloss: 0.0146628
[593]	valid_0's binary_logloss: 0.0146658
[594]	valid_0's binary_logloss: 0.0146522
[595]	valid_0's binary_logloss: 0.0146622
[596]	valid_0's binary_logloss: 0.0146651
[597]	valid_0's binary_logloss: 0.0146733
[598]	valid_0's binary_logloss: 0.0146656
[599]	valid_0's binary_logloss: 0.0146636
[600]	valid_0's binary_logloss: 0.0146748
[601]	valid_0's binary_logloss: 0.0146853
[602]	valid_0's binary_logloss: 0.0146731
[603]	valid_0's binary_logloss: 0.0146424
[604]	valid_0's binary_logloss: 0.0146302
[605]	valid_0's binary_logloss: 0.0146355
[606]	valid_0's binary_logloss: 0.0146311
[607]	valid_0's binary_logloss: 0.0146215
[608]	valid_0's binary_logloss: 0.014624
[609]	valid_0's binary_logloss: 0.0146472
[610]	valid_0's binary_logloss: 0.014636
[611]	valid_0's binary_logloss: 0.0146117
[612]	valid_0's binary_logloss: 0.014594
[613]	valid_0's binary_logloss: 0.0145531
[614]	valid_0's binary_logloss: 0.0145748
[615]	valid_0's binary_logloss: 0.014

In [22]:
y_p = model_m.predict(merge_test_x[:,:100+2351-96])
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, max_test_label)

loss : 0.05746
auc score : 0.99817
accuracy score : 0.98106
thre: 0.9888973644
fp:  0.0009848283
recall:  0.8632343904


(0.9981650125521183, 0.05746487761447228, 0.8632343904459109)

In [23]:
model_t = get_model(merge_train_x[:,100:], max_train_label, merge_val_x[:,100:], max_val_label )

[1]	valid_0's binary_logloss: 0.585956
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.524512
[3]	valid_0's binary_logloss: 0.47255
[4]	valid_0's binary_logloss: 0.429085
[5]	valid_0's binary_logloss: 0.391782
[6]	valid_0's binary_logloss: 0.358682
[7]	valid_0's binary_logloss: 0.330118
[8]	valid_0's binary_logloss: 0.30549
[9]	valid_0's binary_logloss: 0.282788
[10]	valid_0's binary_logloss: 0.262999
[11]	valid_0's binary_logloss: 0.244561
[12]	valid_0's binary_logloss: 0.22845
[13]	valid_0's binary_logloss: 0.214385
[14]	valid_0's binary_logloss: 0.201323
[15]	valid_0's binary_logloss: 0.189575
[16]	valid_0's binary_logloss: 0.179587
[17]	valid_0's binary_logloss: 0.16962
[18]	valid_0's binary_logloss: 0.161338
[19]	valid_0's binary_logloss: 0.153084
[20]	valid_0's binary_logloss: 0.145796
[21]	valid_0's binary_logloss: 0.139744
[22]	valid_0's binary_logloss: 0.13396
[23]	valid_0's binary_logloss: 0.128681
[24]	valid_0's binary_logloss: 0

[199]	valid_0's binary_logloss: 0.021443
[200]	valid_0's binary_logloss: 0.0213981
[201]	valid_0's binary_logloss: 0.0213591
[202]	valid_0's binary_logloss: 0.0212672
[203]	valid_0's binary_logloss: 0.0211935
[204]	valid_0's binary_logloss: 0.0210976
[205]	valid_0's binary_logloss: 0.0210232
[206]	valid_0's binary_logloss: 0.0209166
[207]	valid_0's binary_logloss: 0.0208736
[208]	valid_0's binary_logloss: 0.02076
[209]	valid_0's binary_logloss: 0.0207311
[210]	valid_0's binary_logloss: 0.0206424
[211]	valid_0's binary_logloss: 0.0205676
[212]	valid_0's binary_logloss: 0.0205113
[213]	valid_0's binary_logloss: 0.0204498
[214]	valid_0's binary_logloss: 0.0203973
[215]	valid_0's binary_logloss: 0.0203363
[216]	valid_0's binary_logloss: 0.0202619
[217]	valid_0's binary_logloss: 0.0201873
[218]	valid_0's binary_logloss: 0.0201314
[219]	valid_0's binary_logloss: 0.0200799
[220]	valid_0's binary_logloss: 0.0199886
[221]	valid_0's binary_logloss: 0.0199328
[222]	valid_0's binary_logloss: 0.019

[395]	valid_0's binary_logloss: 0.0136386
[396]	valid_0's binary_logloss: 0.0136352
[397]	valid_0's binary_logloss: 0.013627
[398]	valid_0's binary_logloss: 0.0136012
[399]	valid_0's binary_logloss: 0.0135917
[400]	valid_0's binary_logloss: 0.0135856
[401]	valid_0's binary_logloss: 0.013574
[402]	valid_0's binary_logloss: 0.0135828
[403]	valid_0's binary_logloss: 0.0135756
[404]	valid_0's binary_logloss: 0.0135811
[405]	valid_0's binary_logloss: 0.0135521
[406]	valid_0's binary_logloss: 0.0135475
[407]	valid_0's binary_logloss: 0.0135488
[408]	valid_0's binary_logloss: 0.013535
[409]	valid_0's binary_logloss: 0.0135221
[410]	valid_0's binary_logloss: 0.013512
[411]	valid_0's binary_logloss: 0.0135056
[412]	valid_0's binary_logloss: 0.0135053
[413]	valid_0's binary_logloss: 0.0134987
[414]	valid_0's binary_logloss: 0.0135022
[415]	valid_0's binary_logloss: 0.0134959
[416]	valid_0's binary_logloss: 0.0134572
[417]	valid_0's binary_logloss: 0.0134277
[418]	valid_0's binary_logloss: 0.0134

In [24]:
y_p = model_t.predict(merge_test_x[:,100:])
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, max_test_label)

loss : 0.05525
auc score : 0.99812
accuracy score : 0.98196
thre: 0.9834881903
fp:  0.0009759560
recall:  0.8613909001


(0.9981159760489502, 0.05525050625772819, 0.8613909001041973)

In [25]:
model_t = get_model(merge_train_x[:,100:100+2351-96], max_train_label, merge_val_x[:,100:100+2351-96], max_val_label )

[1]	valid_0's binary_logloss: 0.586016
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.524557
[3]	valid_0's binary_logloss: 0.472536
[4]	valid_0's binary_logloss: 0.429194
[5]	valid_0's binary_logloss: 0.391943
[6]	valid_0's binary_logloss: 0.358771
[7]	valid_0's binary_logloss: 0.329921
[8]	valid_0's binary_logloss: 0.305426
[9]	valid_0's binary_logloss: 0.28325
[10]	valid_0's binary_logloss: 0.263219
[11]	valid_0's binary_logloss: 0.245376
[12]	valid_0's binary_logloss: 0.229147
[13]	valid_0's binary_logloss: 0.214037
[14]	valid_0's binary_logloss: 0.201117
[15]	valid_0's binary_logloss: 0.189759
[16]	valid_0's binary_logloss: 0.178848
[17]	valid_0's binary_logloss: 0.170233
[18]	valid_0's binary_logloss: 0.161162
[19]	valid_0's binary_logloss: 0.15308
[20]	valid_0's binary_logloss: 0.146278
[21]	valid_0's binary_logloss: 0.139942
[22]	valid_0's binary_logloss: 0.133774
[23]	valid_0's binary_logloss: 0.128402
[24]	valid_0's binary_logloss

[199]	valid_0's binary_logloss: 0.0219661
[200]	valid_0's binary_logloss: 0.0218873
[201]	valid_0's binary_logloss: 0.021735
[202]	valid_0's binary_logloss: 0.0216648
[203]	valid_0's binary_logloss: 0.021619
[204]	valid_0's binary_logloss: 0.0215588
[205]	valid_0's binary_logloss: 0.021435
[206]	valid_0's binary_logloss: 0.0213714
[207]	valid_0's binary_logloss: 0.0212448
[208]	valid_0's binary_logloss: 0.0211786
[209]	valid_0's binary_logloss: 0.0211138
[210]	valid_0's binary_logloss: 0.021079
[211]	valid_0's binary_logloss: 0.0209838
[212]	valid_0's binary_logloss: 0.0209088
[213]	valid_0's binary_logloss: 0.0208555
[214]	valid_0's binary_logloss: 0.0208004
[215]	valid_0's binary_logloss: 0.0207257
[216]	valid_0's binary_logloss: 0.0206545
[217]	valid_0's binary_logloss: 0.0205111
[218]	valid_0's binary_logloss: 0.0204422
[219]	valid_0's binary_logloss: 0.0203974
[220]	valid_0's binary_logloss: 0.0203532
[221]	valid_0's binary_logloss: 0.0202602
[222]	valid_0's binary_logloss: 0.0202

[395]	valid_0's binary_logloss: 0.0138455
[396]	valid_0's binary_logloss: 0.0138232
[397]	valid_0's binary_logloss: 0.0138373
[398]	valid_0's binary_logloss: 0.0138382
[399]	valid_0's binary_logloss: 0.0138464
[400]	valid_0's binary_logloss: 0.0138288
[401]	valid_0's binary_logloss: 0.0138294
[402]	valid_0's binary_logloss: 0.0138033
[403]	valid_0's binary_logloss: 0.0137849
[404]	valid_0's binary_logloss: 0.0137716
[405]	valid_0's binary_logloss: 0.0137595
[406]	valid_0's binary_logloss: 0.013735
[407]	valid_0's binary_logloss: 0.0137112
[408]	valid_0's binary_logloss: 0.0136774
[409]	valid_0's binary_logloss: 0.0136813
[410]	valid_0's binary_logloss: 0.0136769
[411]	valid_0's binary_logloss: 0.013666
[412]	valid_0's binary_logloss: 0.0136324
[413]	valid_0's binary_logloss: 0.0136193
[414]	valid_0's binary_logloss: 0.0136206
[415]	valid_0's binary_logloss: 0.0136098
[416]	valid_0's binary_logloss: 0.013584
[417]	valid_0's binary_logloss: 0.0135856
[418]	valid_0's binary_logloss: 0.013

In [26]:
y_p = model_t.predict(merge_test_x[:,100:100+2351-96])
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, max_test_label)

loss : 0.05467
auc score : 0.99807
accuracy score : 0.98184
thre: 0.9842045701
fp:  0.0009848283
recall:  0.8450533009


(0.9980657555692841, 0.05466731261291541, 0.8450533009164017)